# Imports,  Settings and Paths

In [1]:
comment = 'no_batchnorm_096_3'

In [2]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import torch
import torch.utils.data as data_utils

In [5]:
from Code.Models.DeepPotential.deep_potential import DeepPotential
from Code.Train.trainer import Trainer
from Code.Tools.Network.lr_scheduler import AbcExponentialLR

In [ ]:
data_path = './Dataset/c7o2h10_X.npy'
label_path = './Dataset/c7o2h10_Y.npy'
train_ids_path = './Dataset/iso17/train_ids.txt'
test_ids_path = './Dataset/iso17/validation_ids.txt'
eval_path = './evaluation/c7o2h10'

In [ ]:
use_cuda = torch.cuda.is_available()
print('use cuda:  ', use_cuda)

# Read the Data

In [ ]:
train_ids = (np.loadtxt(train_ids_path) - 1).astype(int).tolist()
test_ids = (np.loadtxt(test_ids_path) - 1).astype(int).tolist()

In [ ]:
X_npy = np.load(data_path)
Y_npy = np.load(label_path) * -1

# Train Model

## Instanciate and create Dataloaders

In [ ]:
model = DeepPotential(use_cuda=use_cuda,
                      eval_path=eval_path,
                      comment=comment)
network = Network(model)

In [ ]:
network.create_dataloaders(X_npy, Y_npy)

## Do the learning

In [ ]:
network.fit(350)

In [ ]:
mae = network.calculate_test_mae()
mae

In [ ]:
network.model.train()
latent, labels = [], []
for x, y in network.test_loader:
    if network.model.use_cuda:
        x = x.cuda()
    pred = network.model(x)
    latent.append(pred.detach().cpu())
    labels.append(y)
latent = torch.cat(latent).data
labels = torch.cat(labels).data
latent = backtransform(latent, network.y_min, network.y_max).squeeze()
labels = backtransform(labels, network.y_min, network.y_max).squeeze()
abs(latent - labels).mean().numpy().item()

In [ ]:
network.show_loss_plot()

## Load Model

In [ ]:
test_model = DeepPotential().cuda()
test_model.load_state_dict(torch.load('ModelCheckpoints/c7o2h10/epoch_280'))

In [ ]:
network.model.children()